In [ ]:
!pip install datasets
!pip install torch==1.5.1 torchvision==0.6.1
!pip3 install emoji==0.6.0

from google.colab import drive
drive.mount('/gdrive/')
%cd /gdrive/MyDrive/ML

import torch

# from numba import cuda
# device = cuda.get_current_device()
# device.reset()

import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 8.2 MB/s 
     |████████████████████████████████| 115 kB 60.3 MB/s 
     |████████████████████████████████| 212 kB 65.2 MB/s 
     |████████████████████████████████| 141 kB 61.4 MB/s 
     |████████████████████████████████| 127 kB 57.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 753.2 MB 13 kB/s 
     |████████████████████████████████| 6.6 MB 33.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))# df.set_index('id', inplace=True)
# 
df = pd.read_csv(r"/gdrive/MyDrive/ML/tweet_emotions.csv")

In [ ]:
#we want to remove small datasets 
# df = df[~df.sentiment.str.contains('\|')]   
df = df[df.sentiment != 'anger'] #& 'boredom' & 'enthusiasm' & 'empty'
df = df[df.sentiment != 'boredom']
df = df[df.sentiment != 'enthusiasm']
df = df[df.sentiment != 'empty']
df = df[df.sentiment != 'sentiment'] #there is sentiment in sentiments!
df = df[df.sentiment != 'fun']
df = df[df.sentiment != 'relief']
df = df[df.sentiment != 'hate']
df = df[df.sentiment != 'surprise']

df.sentiment.value_counts()
#class imbalance

neutral      8638
worry        8459
happiness    5209
sadness      5165
love         3842
Name: sentiment, dtype: int64

In [ ]:
#build dictionary, key: emotion, value: 
possible_labels = df.sentiment.unique()

label_dict = {}
#loop over index
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
print(label_dict)

{'sadness': 0, 'neutral': 1, 'worry': 2, 'love': 3, 'happiness': 4}


In [ ]:
print(df)

         tweet_id  sentiment  \
1      1956967666    sadness   
2      1956967696    sadness   
4      1956968416    neutral   
5      1956968477      worry   
6      1956968487    sadness   
...           ...        ...   
39995  1753918954    neutral   
39996  1753919001       love   
39997  1753919005       love   
39998  1753919043  happiness   
39999  1753919049       love   

                                                 content  
1      Layin n bed with a headache  ughhhh...waitin o...  
2                    Funeral ceremony...gloomy friday...  
4      @dannycastillo We want to trade with someone w...  
5      Re-pinging @ghostridah14: why didn't you go to...  
6      I should be sleep, but im not! thinking about ...  
...                                                  ...  
39995                                   @JohnLloydTaylor  
39996                     Happy Mothers Day  All my love  
39997  Happy Mother's Day to all the mommies out ther...  
39998  @niariley WASSUP B

In [ ]:
from datasets import Value, ClassLabel, Features, Dataset

#build new column for these values
df["label"] = df.sentiment.replace(label_dict)

features = Features({"label": Value("int64"), "label": ClassLabel(num_classes=5, names=[0,1,2,3,4])})
# ds = Dataset.from_pandas(df, features=features)
# ds.features

In [ ]:
from sklearn.model_selection import train_test_split



#stratified split
X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                 df.label.values,
                                                 test_size = 0.025,
#                                                  random_state=17,
                                                 stratify = df.label.values
                                                 )



df['data_type'] = ['not_set']*df.shape[0]

df.head()

,tweet_id,sentiment,content,label,data_type
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...,0,not_set
2,1956967696,sadness,Funeral ceremony...gloomy friday...,0,not_set
4,1956968416,neutral,@dannycastillo We want to trade with someone w...,1,not_set
5,1956968477,worry,Re-pinging @ghostridah14: why didn't you go to...,2,not_set
6,1956968487,sadness,"I should be sleep, but im not! thinking about ...",0,not_set


In [ ]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['sentiment', 'label', 'data_type']).count()
#group by using count

tweet_id  content
sentiment label data_type                   
happiness 4     train          5079     5079
                val             130      130
love      3     train          3746     3746
                val              96       96
neutral   1     train          8422     8422
                val             216      216
sadness   0     train          5036     5036
                val             129      129
worry     2     train          8247     8247
                val             212      212

In [ ]:
# Install spaCy (run in terminal/prompt)
import sys
!{sys.executable} -m pip install spacy

# Download spaCy's  'en' Model
!{sys.executable} -m spacy download en_core_web_sm

!pip install -U symspellpy

#for spell and slang correction
!pip install gingerit
from gingerit.gingerit import GingerIt

#for emoticons
!pip install emot --upgrade
import emot 
emot_obj = emot.core.emot() 

from symspellpy.symspellpy import SymSpell, Verbosity
import pkg_resources
import re, string, json
import spacy

contraction_list = json.loads(open(r"/gdrive/MyDrive/ML/english_contractions.json.txt", 'r').read())
character_entity= {'&lt;3':'heart', '&amp;':'and','&quot;':' quote '}
contraction_list = {**contraction_list, **character_entity}


def normalization_pipeline(sentences):
    print("##############################")
    print("Starting Normalization Process")
    sentences = _simplify_punctuation_and_whitespace(sentences) # !!!!! "      "
    sentences = _normalize_contractions(sentences) #also corrects spelling now
    print("Normalization Process Finished")
    print("##############################")
    return sentences

    
def _simplify_punctuation_and_whitespace(sentence_list):
    """
    words with more than 4 all-capital words will get <-EMPW 
    """
    norm_sents = []
    print("Replacing -URL- , Replacing @MENTION and #HASHTAG, Reducing character repetitions, ")
    print("Simplifying punctuation, Removing whitespaces")

    for sentence in tqdm(sentence_list):
        sent = _replace_urls(sentence)
        sent = _mention_hash(sent)
        sent = _simplify_punctuation(sent)
        sent = _reduce_repetitions(sent)
        sent = _normalize_whitespace(sent)
        norm_sents.append(sent)
    return norm_sents


def _replace_urls(text):
    url_regex = r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'
    text = re.sub(url_regex, "-URL-", text)
    return text


def _mention_hash(in_str):
    """
     @MENTIONs and #HASHTAGs will take forms of @men and #has 
    note: BEWARE OF USES OF # AND @ AND SPACES BETWEEN THEM
    """
    in_str = str(in_str)
    in_str = re.sub('@\w+', '@MEN', in_str,flags=re.IGNORECASE) # use @\w+ for word replacement or @ with space after @MEN for keeping mention
    in_str = re.sub('#', '#HAS ', in_str,flags=re.IGNORECASE)
    in_str = re.sub(r'([\w])\1+', r'\1\1', in_str) #reduce repeated characters to 2
    return in_str.strip()


def _simplify_punctuation(text):
    """
    puntuations like '!!!!!' will be transformed into '!! <-EMPP'
    This function simplifies doubled or more complex punctuation. The exception is '...'. #?! ??? !!!
    """
    corrected = str(text)
    corrected = re.sub(r'([!?,;])\1+', r'\1\1 <-EMPP', corrected) #\1\1 makes it to 2 consecutive punctuation
    corrected = re.sub(r'\.{2,}', r'...', corrected)
    return corrected

def _reduce_repetitions(text):
    """
    Auxiliary function to help with exxagerated (repeated characters in) words.
    Examples:
        woooooords -> woords <-EMPW
        dooorwaaay -> doorwaay <-EMPW
        SICK -> sick <-EMPU
    """
    correction = str(text)
    for index, words in enumerate(str(text).split()):
        if _is_EMP_word(words)==True :
            #insert EMPW after word
            correction = correction.replace(words, words + ' <-EMPW')
        if (len(words) > 4) & (words.isupper()==True) & (words[0] not in string.punctuation):
            correction = correction.replace(words, words + ' <-EMPU')
    #TODO work on complexity reduction.
    return re.sub(r'([\w])\1+', r'\1\1', correction) #\1\1 will only keep 2 consecutive characters


def _is_EMP_word(word):
    """
    True/ False: checks if the word has 3 consecutive characters"""
    count=1
    if len(word)>1:
        for i in range(1,len(word)):
            if word[i] in string.punctuation: #this function is only for words!
                return False
            if word[i-1]==word[i]:
                count+=1
                if(count>=3):
                     return True
            else :
                if(count>=3):
                    return True
                count=1
    else :
        return False
    return False


def _normalize_whitespace(text):
    """
    normalizes whitespaces, removing duplicates.
    """
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")
    
    
#Substitution of contractions:  -----------------------------------------------------------------------------------------------      
def _normalize_contractions(sentence_list):
    """
    it will correct each word in a sentence for slangs(ginger), emojis -> meaning, entity references and abbreviations(json file) : file can be manually modified above
    also makes everything lowercase (including EMPW,EMPU, EMPP, URL, etc)
    """
    #uses contraction_list (a json file) BE SURE TO IMPORT IT ALREADY
    norm_sents = []
    print("Normalizing contractions, abbreviations, slangs, emojis, character entities")
    for sentence in tqdm(sentence_list):
        norm_sents.append(_normalize_contractions_slang_emoji_entity(sentence))
    return norm_sents

def _normalize_contractions_slang_emoji_entity(text):
    """
    part1:normalizes english contractions.
    """
    contractions = contraction_list
    for word in text.split():
         if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])
#             print('replacing contraction: '+ word + ' to '+contractions[word.lower()])
    """
    part 2: using gingerit SMS slang correction:
    this is too slow and can take many hours for the whole dataset to run
    """
#     parser = GingerIt()
#     result=parser.parse(text)
#     # corrections = result['corrections']
#     sentence = result['result']
    sentence = text
    """
    part3: emoji and character entity reference conversion to meaning
    """
#     if emot_obj.emoji(sentence)['value'] !=[] : #we do not have emojis in this database text
#         print("found emoji: "+str(emot_obj.emoji(sentence)['value'])+ sentence)
    emoticons = emot_obj.emoticons(sentence)
#     if((emoticons['value']!=[]) ): #for printing
#         print("found: "+str(emoticons['value']) +'  emoticons in:   '+ sentence) 
    for i in range(0,len(emoticons['value'])):
#         print('replacing  ' + emoticons['value'][i] + '  with ' +  emoticons['mean'][i])
        sentence = sentence.replace(emoticons['value'][i], emoticons['mean'][i])
    """
    part4: make everything lowercase
    """
    sentence = sentence.lower()
    return sentence

# df = pd.read_csv(r"/gdrive/MyDrive/ML/english_contractions.json.txt")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 5.1 MB/s 
     |████████████████████████████████| 125 kB 59.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 15 kB/s 


In [ ]:
print(df)

         tweet_id  sentiment  \
1      1956967666    sadness   
2      1956967696    sadness   
4      1956968416    neutral   
5      1956968477      worry   
6      1956968487    sadness   
...           ...        ...   
39995  1753918954    neutral   
39996  1753919001       love   
39997  1753919005       love   
39998  1753919043  happiness   
39999  1753919049       love   

                                                 content  label data_type  
1      Layin n bed with a headache  ughhhh...waitin o...      0     train  
2                    Funeral ceremony...gloomy friday...      0     train  
4      @dannycastillo We want to trade with someone w...      1     train  
5      Re-pinging @ghostridah14: why didn't you go to...      2     train  
6      I should be sleep, but im not! thinking about ...      0     train  
...                                                  ...    ...       ...  
39995                                   @JohnLloydTaylor      1     train  
39996  

In [ ]:
#assessment and examples:
# original_examples = ['hi @someone WATCH me #proud :) ;) ...... i h8 it bt w8 !!!!!  <3  wanna go &amp; &lt;3 tHeRe  &quot; bcs my finls clooooose &quot;bananas&quot; &amp; ']
original_examples=df.content[0:10]
preprocessed_examples = normalization_pipeline(original_examples)
for example_index,example in enumerate(preprocessed_examples):
#     print(original_examples[example_index])
    print(original_examples.values[example_index])
    print(example)


    
#run preprocessing
df_original=df
df.content=normalization_pipeline(df.content.values ) #about 10 minutes to run

#save
df.to_csv('df_processed.csv',index=False)

#load
df = pd.read_csv(r"/gdrive/MyDrive/ML/english_contractions.json.txt")



##############################
Starting Normalization Process
Replacing -URL- , Replacing @MENTION and #HASHTAG, Reducing character repetitions, 
Simplifying punctuation, Removing whitespaces


  0%|          | 0/10 [00:00<?, ?it/s]

Normalizing contractions, abbreviations, slangs, emojis, character entities


  0%|          | 0/10 [00:00<?, ?it/s]

Normalization Process Finished
##############################
Layin n bed with a headache  ughhhh...waitin on your call...
layin n bed with a headache ughh...waitin on your call...
Funeral ceremony...gloomy friday...
funeral ceremony...gloomy friday...
@dannycastillo We want to trade with someone who has Houston tickets, but no one will.
@men we want to trade with someone who has houston tickets, but no one will.
Re-pinging @ghostridah14: why didn't you go to prom? BC my bf didn't like my friends
re-pinging @men: why did not you go to prom? bc my bf did not like my friends
I should be sleep, but im not! thinking about an old friend who I want. but he's married now. damn, &amp; he wants me 2! scandalous!
i should be sleep, but i am not! thinking about an old friend who i want. but he has married now. damn, and he wants me 2! scandalous!
Hmmm. http://www.djhero.com/ is down
hmm. -url- is down
@charviray Charlene my love. I miss you
@men charlene my love. i miss you
@kelcouch I'm sorry  a

  0%|          | 0/31313 [00:00<?, ?it/s]

Normalizing contractions, abbreviations, slangs, emojis, character entities


  0%|          | 0/31313 [00:00<?, ?it/s]

Normalization Process Finished
##############################


In [ ]:
df_processed = pd.read_csv(r"df_processed.csv")
print(df_processed)

         tweet_id  sentiment  \
0      1956967666    sadness   
1      1956967696    sadness   
2      1956968416    neutral   
3      1956968477      worry   
4      1956968487    sadness   
...           ...        ...   
31308  1753918954    neutral   
31309  1753919001       love   
31310  1753919005       love   
31311  1753919043  happiness   
31312  1753919049       love   

                                                 content  label data_type  
0      layin n bed with a headache ughh...waitin on y...      0     train  
1                    funeral ceremony...gloomy friday...      0     train  
2      @men we want to trade with someone who has hou...      1     train  
3      re-pinging @men: why did not you go to prom? b...      2     train  
4      i should be sleep, but i am not! thinking abou...      0     train  
...                                                  ...    ...       ...  
31308                                               @men      1     train  
31309  

In [ ]:
spacy.load('en_core_web_sm')

In [ ]:
# import matplotlib.pyplot as plt
# from collections import Counter


# tokenizee=[]
# for words in tqdm(range(1,len(df_processed.content)-1)):
#     tokenizee.append(spacy_process(df_processed.content[words]))
    
# #overview of preprocessed data
# words = Counter()
# for s in tokenizee:
#   for w in s:
#     words[w] += 1

# sorted_words = list(words.keys())
# sorted_words.sort(key=lambda w: words[w], reverse=True)
# print(f"Number of different Tokens in our Dataset: {len(sorted_words)}")
# print(sorted_words[:100])


# count_occurences = sum(words.values())
# accumulated = 0
# counter = 0
# while accumulated < count_occurences * 0.8:
#   accumulated += words[sorted_words[counter]]
#   counter += 1

# print(f"The {counter * 100 / len(words)}% most common words "
#       f"account for the {accumulated * 100 / count_occurences}% of the occurrences")

# plt.bar(range(100), [words[w] for w in sorted_words[:100]])
# plt.show()

In [ ]:
from datasets import Dataset
import pandas as pd
dataset = Dataset.from_pandas(df_processed)

dataset = dataset.rename_column('content', "text")
dataset = dataset.rename_column('tweet_id', "id")

In [3]:
!pip install transformers
from transformers import AutoTokenizer

#Loads model and tokenizes stuff
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base") #Replace "" with relevant model

#Defines function for pre-processing - e.g. tokenising and truncating long phrases
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

#Maps pre-processing function to all of dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.8 MB/s 
     |████████████████████████████████| 101 kB 10.3 MB/s 
     |████████████████████████████████| 596 kB 50.7 MB/s 
     |████████████████████████████████| 6.6 MB 42.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


NameError: ignored

In [ ]:
print(dataset[2])

{'id': 1956968416, 'sentiment': 'neutral', 'text': '@men we want to trade with someone who has houston tickets, but no one will.', 'label': 1, 'data_type': 'train'}


In [ ]:
from datasets import load_metric
import numpy as np

metric = load_metric("accuracy")

def compute_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [2]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=5) 
#Change model names and number of classes as is relevant.

training_args = TrainingArguments(
    #gradient_accumulation_steps=2,
    #gradient_checkpointing=True,
    #Memory-saving, more efficient way of reviewing dataset entries. Multiply by batch size for effective batch size.
    output_dir="./results",
    learning_rate=5e-5,
    per_device_train_batch_size=32, #Decrease to save memory, but start at 16
    per_device_eval_batch_size=32, #Decrease to save memory, but start at 16
    optim="adafactor", #Optimiser - add to save memory
    num_train_epochs=6, #Start at five, but alter based on optimum loss
    weight_decay=0.01,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset.filter(lambda x: x['data_type'] =='train'),
    eval_dataset=tokenized_dataset.filter(lambda x: x['data_type'] =='val'),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_accuracy, #Call accuracy score function
)

ModuleNotFoundError: ignored

In [ ]:
print(tokenized_dataset[200])

{'id': 1957019458, 'sentiment': 'neutral', 'text': '#has 3wordsaftersex it never started...', 'label': 1, 'data_type': 'train', 'input_ids': [0, 85, 90, 698, 25844, 37734, 1871, 557, 18, 143, 4469, 13978, 28, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: id, data_type, sentiment, text. If id, data_type, sentiment, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 30530
  Num Epochs = 6
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 5730


Step,Training Loss
500,1.263300
1000,1.201600
1500,1.097900
2000,1.067900
2500,0.942600
3000,0.885100
3500,0.747500
4000,0.688700
4500,0.583300
5000,0.525300


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
added tokens file saved in ./results/checkpoint-500/added_tokens.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
added tokens file saved in ./results/checkpoint-1000/added_tokens.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
token

TrainOutput(global_step=5730, training_loss=0.8422834879024699, metrics={'train_runtime': 2004.6301, 'train_samples_per_second': 91.378, 'train_steps_per_second': 2.858, 'total_flos': 4057379398293060.0, 'train_loss': 0.8422834879024699, 'epoch': 6.0})

In [ ]:
torch.save(model.state_dict(),'mod2.pt')

In [1]:
model.load_state_dict(torch.load('mod2.pt'))

NameError: ignored

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: id, data_type, sentiment, text. If id, data_type, sentiment, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 783
  Batch size = 32


{'epoch': 6.0,
 'eval_accuracy': 0.45849297573435505,
 'eval_loss': 1.801609754562378,
 'eval_runtime': 2.2789,
 'eval_samples_per_second': 343.589,
 'eval_steps_per_second': 10.97}

In [ ]:
test = {'text': ["it was a nice day out so im glad you enjoyed it too"]}

test = preprocess_function(test)

test['input_ids'] = torch.tensor(test['input_ids'])
test['attention_mask'] = torch.tensor(test['attention_mask'])

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
test['input_ids'] = test['input_ids'].to(device) 
test['attention_mask'] = test['attention_mask'].to(device)

print("predictions:", model(test['input_ids'], test['attention_mask']))

predictions: SequenceClassifierOutput(loss=None, logits=tensor([[-2.1222, -1.6602, -0.3510,  0.3700,  3.9351]], device='cuda:0',
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)


In [ ]:
#'sadness': 0, 'neutral': 1, 'worry': 2, 'love': 3, 'happiness': 4